In [4]:
!pip install transformers

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


In [ ]:
# Simple example
text = "The small dog walked around the park happily with its owner and "
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate 5 words, one at a time
current_text = text
for step in range(30):
    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")
    
    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")
    
    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits
    
    # Get the predictions for the NEXT token
    next_token_logits = predictions[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=0)
    
    # Get top 5 predictions
    top_probs, top_indices = torch.topk(next_token_probs, 10)
    
    print("Top 5 next word predictions:")
    for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
        word = tokenizer.decode([idx])
        print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")
    
    # Top K pred
    # top_probs = top_probs / top_probs.sum()
    # sampled_index = torch.multinomial(top_probs, 1)
    # next_token_id = top_indices[sampled_index]
    # next_word = tokenizer.decode(next_token_id)
    # current_text += next_word

    # Use the most likely word (greedy)
    next_token_id = top_indices[0]
    next_word = tokenizer.decode([next_token_id])
    current_text += next_word
    
    print(f"✓ Chosen: '{next_word}'")
    print(f"New text: '{current_text}'\n")

print(f"\nFinal generated text: '{current_text}'")


Starting text: 'The small dog walked around the park happily with its owner and '

Generating word by word...

--- Step 1 ---
Current: 'The small dog walked around the park happily with its owner and '
Top 5 next word predictions:
  1. 'urch' (24.9% confident)
  2. 'iced' (11.6% confident)
  3. ' ' (10.9% confident)
  4. 'ute' (8.2% confident)
  5. 'ers' (3.5% confident)
  6. 'vern' (3.1% confident)
  7. 'urs' (2.6% confident)
  8. 'urn' (2.0% confident)
  9. 'iba' (1.4% confident)
  10. 'ik' (1.3% confident)
✓ Chosen: 'urch'
New text: 'The small dog walked around the park happily with its owner and urch'

--- Step 2 ---
Current: 'The small dog walked around the park happily with its owner and urch'
Top 5 next word predictions:
  1. 'in' (82.5% confident)
  2. 'ins' (14.9% confident)
  3. 'ased' (1.3% confident)
  4. 'aser' (0.3% confident)
  5. 'ired' (0.1% confident)
  6. 'ined' (0.1% confident)
  7. 'asers' (0.1% confident)
  8. 'and' (0.1% confident)
  9. 'as' (0.0% confident)
  1